In [1]:
import pandas as pd
import numpy as np

In [ ]:
def collectCDSRegions():
    clash = pd.read_csv("../CLASH-mmc1.txt", sep="\s+", comment="#")
    clash = clash[["microRNA_name", "miRNA_seq", "mRNA_name", "mRNA_seq_extended", "seed_type", "5'UTR", "CDS", "3'UTR"]]
    
    clash = clash.dropna(axis=0, subset=["CDS"])
    clash = clash[~(clash["5'UTR"]==1.0)]
    clash = clash[~(clash["3'UTR"]==1.0)]
    print(clash.seed_type.value_counts())
    clash = clash.drop(["5'UTR", "CDS", "3'UTR"], axis=1)
    
    """Get MIMAT id and seed family"""
    clash["mimat_name"] = clash["microRNA_name"].str.extract(r"(MIMAT\d*)")
    clash["seed_family"] = clash["microRNA_name"].str.extract(r"((?<=MirBase_).*(?=_))")    
    
    
    clash.to_csv("CLASH-cdsOnly.txt", sep="\t", index=False)

In [2]:
# collectCDSRegions()

In [3]:
import random

In [22]:
def mismatchCLASHSequences():
    """Mismatch miRNA-mRNA pairs such that the new miRNA match is not from the same seed family as the original"""
    """Also use miRBase miRNA"""
    
    clash = pd.read_csv("CLASH-cdsOnly.txt", sep="\t")
    clash["miRNA_seq"] = clash["miRNA_seq"].str.replace("T", "U")
    clash["mRNA_seq_extended"] = clash["mRNA_seq_extended"].str.replace("T", "U")
    
    mirbase = pd.read_csv("../mirbase_v22.1_seqs.txt", sep="\t")
    mirbase = mirbase[["mimat_name", "mirna_seq"]]
    print(mirbase.shape)

    clash_mirbase = clash.merge(mirbase, how="inner", on="mimat_name")
    pos_dataset = clash_mirbase[["mimat_name", "seed_family", "mirna_seq", "mRNA_seq_extended"]].copy()
    pos_dataset["classLabel"] = 1
#     pos_dataset.to_csv("positive_dataset.txt", sep="\t", index=False)
    
    """Create dictionary mirna_seq:mimat_name, mimat_name:seed_family"""
#     seq_features = pos_dataset.drop_duplicates(subset=["mimat_name", "mirna_seq"])
#     seq_features = seq_features[["mimat_name", "seed_family", "mirna_seq"]]
#     print(clash_mirbase.mimat_name.value_counts())
#     mimat_to_seq = {k:v for k, v in zip(seq_features["mimat_name"].tolist(), seq_features["mirna_seq"].tolist())}
    
    """Mismatch binding pairs"""
    pos_dataset = pd.read_csv("positive_dataset.txt", sep="\t")
    neg_dataset = pd.read_csv("positive_dataset.txt", sep="\t")
    neg_dataset["original_mirna_seq"] = pos_dataset["mirna_seq"].copy()
    neg_dataset["original_mimat_name"] = pos_dataset["mimat_name"].copy()
    neg_dataset["original_seed_family"] = pos_dataset["seed_family"].copy()
    
#     print(neg_dataset)
#     print((pos_dataset["mRNA_seq_extended"]==neg_dataset["mRNA_seq_extended"]).all())
    for idx, row in neg_dataset.iterrows():
        random_idx = random.randrange(0, neg_dataset.shape[0])
        pos_pair = pos_dataset[pos_dataset["mRNA_seq_extended"]==row["mRNA_seq_extended"]]
#         print(random_idx, neg_dataset.iloc[random_idx]["original_seed_family"])
#         print(row["seed_family"])
#         while (neg_dataset.iloc[random_idx]["seed_family"]==row["seed_family"]) or 
        while (neg_dataset.iloc[random_idx]["original_seed_family"] in pos_pair["seed_family"].tolist()) or (neg_dataset.iloc[random_idx]["original_mirna_seq"] in pos_pair["mirna_seq"].tolist()):
            random_idx = random.randrange(0, neg_dataset.shape[0])
           
        tmp_seq = row["mirna_seq"]
        tmp_seed = row["seed_family"]
        tmp_mimat = row["mimat_name"]

#         neg_dataset.at[idx, "mirna_seq"] = mimat_to_seq[neg_dataset.iloc[random_idx]["original_mimat_name"]]
        neg_dataset.at[idx, "mirna_seq"] = neg_dataset.iloc[random_idx]["original_mirna_seq"]
        neg_dataset.at[idx, "seed_family"] = neg_dataset.iloc[random_idx]["original_seed_family"]
        neg_dataset.at[idx, "mimat_name"] = neg_dataset.iloc[random_idx]["original_mimat_name"]
           
        neg_dataset.at[random_idx, "mirna_seq"] = tmp_seq
        neg_dataset.at[random_idx, "seed_family"] = tmp_seed
        neg_dataset.at[random_idx, "mimat_name"] = tmp_mimat
        
#         break
    """Match up mirna_seq with their corresponding mimat_name and seed_family"""
    neg_dataset = neg_dataset[["mimat_name", "seed_family", "mirna_seq", "mRNA_seq_extended"]]
    neg_dataset["classLabel"] = 0
    
    neg_dataset.to_csv("negative_dataset.txt", sep="\t", index=False)

In [23]:
mismatchCLASHSequences()

(2656, 2)


In [24]:
def checkNegDataset():
    """Make sure the mirnas are from a different seed family"""
    pos = pd.read_csv("positive_dataset.txt", sep="\t")
    neg = pd.read_csv("negative_dataset_keep.txt", sep="\t")
     
    count=0
    for idx, row in neg.iterrows():
        pos_pair = pos[pos["mRNA_seq_extended"]==row["mRNA_seq_extended"]]
        if (neg.iloc[idx]["seed_family"] in pos_pair["seed_family"].tolist()) or (row["mirna_seq"] in pos_pair["mirna_seq"].tolist()):
#             print(pos[pos["mRNA_seq_extended"]==row["mRNA_seq_extended"]], row["mRNA_seq_extended"])
            count += 1
            print(row["seed_family"], pos_pair["seed_family"].tolist())
            print(idx)
    print(count)

In [25]:
checkNegDataset()

let-7c ['let-7a', 'let-7c']
125
1
